Modified from https://github.com/melanieabrams-pub/RH-seq_with_barcoding/blob/main/annotate_poolfile_for_barseq_v1.2_JSkerker.ipynb and 
https://github.com/melanieabrams-pub/kluyv_RH-seq_with_barcoding/blob/main/annotate_poolfile_kluyv.py

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

### Load GFF annotations

In [2]:
#you don't need to change the path below - this is the most recent annotation file (5/12/22)
gff_file = '/usr2/people/clairedubin/barseq/genome_and_annotations/Klac_Kmar_CDS_orthologs_only.gff'

In [3]:
gff = pd.read_csv(gff_file, sep='\t')

In [4]:
t = gff['scaffold'].unique()
print(t)

['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr10' 'chr11' 'chr12' 'chr13'
 'chr14' 'chr15' 'chr23' 'chr24' 'chr32' 'chr7' 'chr8' 'chr9']


In [5]:
gff.head()

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,YGOB/YGAP,CDS,22024,25698,-,"gene_id ""KLLA0A00286g""; transcript_id ""KLLA0A00286g""",klYJL074C
1,chr1,YGOB/YGAP,CDS,29717,32755,+,"gene_id ""KLLA0A00396g""; transcript_id ""KLLA0A00396g""",klYIL156W;YKR098C
2,chr1,YGOB/YGAP,CDS,33084,35012,-,"gene_id ""KLLA0A00418g""; transcript_id ""KLLA0A00418g""",klYIL155C
3,chr1,YGOB/YGAP,CDS,37134,38201,-,"gene_id ""KLLA0A00440g""; transcript_id ""KLLA0A00440g""",klYIL154C
4,chr1,YGOB/YGAP,CDS,38766,39827,+,"gene_id ""KLLA0A00462g""; transcript_id ""KLLA0A00462g""",klYIL153W


In [6]:
gff

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,YGOB/YGAP,CDS,22024,25698,-,"gene_id ""KLLA0A00286g""; transcript_id ""KLLA0A00286g""",klYJL074C
1,chr1,YGOB/YGAP,CDS,29717,32755,+,"gene_id ""KLLA0A00396g""; transcript_id ""KLLA0A00396g""",klYIL156W;YKR098C
2,chr1,YGOB/YGAP,CDS,33084,35012,-,"gene_id ""KLLA0A00418g""; transcript_id ""KLLA0A00418g""",klYIL155C
3,chr1,YGOB/YGAP,CDS,37134,38201,-,"gene_id ""KLLA0A00440g""; transcript_id ""KLLA0A00440g""",klYIL154C
4,chr1,YGOB/YGAP,CDS,38766,39827,+,"gene_id ""KLLA0A00462g""; transcript_id ""KLLA0A00462g""",klYIL153W
...,...,...,...,...,...,...,...,...
8960,chr9,YGOB/YGAP,CDS,1534846,1535949,-,"gene_id ""KM020C07240""; transcript_id ""KM020C07240""",kmYIL153W
8961,chr9,YGOB/YGAP,CDS,1536581,1537672,+,"gene_id ""KM020C07250""; transcript_id ""KM020C07250""",kmYIL154C
8962,chr9,YGOB/YGAP,CDS,1539622,1541550,+,"gene_id ""KM020C07260""; transcript_id ""KM020C07260""",kmYIL155C
8963,chr9,YGOB/YGAP,CDS,1541776,1544832,-,"gene_id ""KM020C07270""; transcript_id ""KM020C07270""",kmYIL156W;YKR098C


### Match barcode insert locations to gene annotations

In [19]:
!pwd

/auto/sahara/namib/home/kayleeec/rhseq


In [17]:
#CHANGE IF NEEDED: load COMBINED poolfile generated in TNseq mapping step

poolfile = '/usr2/people/kayleeec/rhseq/Test_TNSeq/Test_Output_formatting/POOL30_poolfile'
poolfile = 'Test_TNSeq/Test_Output_formatting/POOL30_poolfile'


#CHANGE IF NEEDED: annotated pool file path to output

annotated_poolfile_path ='/usr2/people/shollyt22/shollyt22/Barseq_July_2023/OORB003_TnSeq/for_analysis/kaylee_file_annotated.csv'

In [18]:
df = pd.read_csv(poolfile)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/usr2/people/kayleeec/rhseq/Test_TNSeq/Test_Output_formatting/POOL30_poolfile'

Check that each barcode in the dataframe is unique (the two numbers below should be the same):

In [42]:
print('Number of insertions in poolfile: {}'.format(df.shape[0]))
#print('Number of unique barcodes in poolfile: {}'.format(len(df['barcode'].unique())))

Number of insertions in poolfile: 2406


In [43]:
df = df.set_index('transposonLocation')

In [24]:
#df.columns

Index(['rcbarcode', 'nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
       'nMainLocation', 'nInsert', 'All genomic mappings',
       'All insert mappings'],
      dtype='object')

In [44]:
df.head()

,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
transposonLocation,,,,,,,,,,
chr9:937375:+,1,1,chr9,+,937375,Single,1,0,"[('chr9:937375:+', 1)]",{'Null:0:+': 0}
chr2:338609:+,1,1,chr2,+,338609,Single,1,0,"[('chr2:338609:+', 1)]",{'Null:0:+': 0}
chr4:413026:+,2,2,chr4,+,413026,Single,2,0,"[('chr4:413026:+', 2)]",{'Null:0:+': 0}
chr14:596883:+,1,1,chr14,+,596883,Single,1,0,"[('chr14:596883:+', 1)]",{'Null:0:+': 0}
chr13:899947:+,1,1,chr13,+,899947,Single,1,0,"[('chr13:899947:+', 1)]",{'Null:0:+': 0}


In [30]:
#df=df.drop(df.columns[-1],axis=1)

In [31]:
#df.columns = ['nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
       'nMainLocation', 'nInsert', 'All genomic mappings',
       'All insert mappings']

In [32]:
#df.head()

,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
barcode,,,,,,,,,,
chr9:937375:+,1,1,chr9,+,937375,Single,1,0,"[('chr9:937375:+', 1)]",{'Null:0:+': 0}
chr2:338609:+,1,1,chr2,+,338609,Single,1,0,"[('chr2:338609:+', 1)]",{'Null:0:+': 0}
chr4:413026:+,2,2,chr4,+,413026,Single,2,0,"[('chr4:413026:+', 2)]",{'Null:0:+': 0}
chr14:596883:+,2,2,chr14,+,596883,Single,2,0,"[('chr14:596883:+', 2)]",{'Null:0:+': 0}
chr13:899947:+,1,1,chr13,+,899947,Single,1,0,"[('chr13:899947:+', 1)]",{'Null:0:+': 0}


In [45]:
#this takes a few minutes
def get_gene_by_bc_position(pos):
    
    gene_list = [scaffold_gff.loc[interval]['ID'] for interval in idx if pos in interval]
    
    #barcode position not in any genes
    if not gene_list:
        return ''
    #barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    #barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    
    scaffold_gff = gff[gff['scaffold']==scaffold]
    scaffold_barcodes = df[df['scaffold']==scaffold]
    
    #divide scaffold_gff.shape by 2 here to get number of genes (original shape includes promoters)
    print('Mapping {} barcodes on {} to {} genes on {}'.format(scaffold_barcodes.shape[0], scaffold,
                                                              round(scaffold_gff.shape[0]/2), scaffold))

    #turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, 1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    #for each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)

Mapping 122 barcodes on chr1 to 218 genes on chr1
Mapping 150 barcodes on chr10 to 282 genes on chr10
Mapping 114 barcodes on chr11 to 248 genes on chr11
Mapping 124 barcodes on chr12 to 229 genes on chr12
Mapping 94 barcodes on chr13 to 196 genes on chr13
Mapping 74 barcodes on chr14 to 179 genes on chr14
Mapping 24 barcodes on chr15 to 40 genes on chr15
Mapping 171 barcodes on chr2 to 283 genes on chr2
Mapping 0 barcodes on chr23 to 2 genes on chr23
Mapping 0 barcodes on chr24 to 1 genes on chr24
Mapping 226 barcodes on chr3 to 373 genes on chr3
Mapping 0 barcodes on chr32 to 1 genes on chr32
Mapping 224 barcodes on chr4 to 356 genes on chr4
Mapping 278 barcodes on chr5 to 500 genes on chr5
Mapping 325 barcodes on chr6 to 535 genes on chr6
Mapping 170 barcodes on chr7 to 365 genes on chr7
Mapping 135 barcodes on chr8 to 357 genes on chr8
Mapping 175 barcodes on chr9 to 318 genes on chr9


In [46]:
#check that a 'gene' column has been added to df

df.head()

,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings,gene
transposonLocation,,,,,,,,,,,
chr9:937375:+,1,1,chr9,+,937375,Single,1,0,"[('chr9:937375:+', 1)]",{'Null:0:+': 0},kmYHR034C
chr2:338609:+,1,1,chr2,+,338609,Single,1,0,"[('chr2:338609:+', 1)]",{'Null:0:+': 0},klYJR144W
chr4:413026:+,2,2,chr4,+,413026,Single,2,0,"[('chr4:413026:+', 2)]",{'Null:0:+': 0},
chr14:596883:+,1,1,chr14,+,596883,Single,1,0,"[('chr14:596883:+', 1)]",{'Null:0:+': 0},
chr13:899947:+,1,1,chr13,+,899947,Single,1,0,"[('chr13:899947:+', 1)]",{'Null:0:+': 0},


In [47]:
print('{} barcodes not in coding sequences'.format(df[df['gene']==''].shape[0]))
print()

print('{} barcodes in coding sequences'.format(df[df['gene']!=''].shape[0]))
print('{} barcodes in Kl coding sequences'.format(df[df['gene'].str.startswith('kl',na=False)].shape[0]))
print('{} barcodes in Km coding sequences'.format(df[df['gene'].str.startswith('km',na=False)].shape[0]))

1861 barcodes not in coding sequences

545 barcodes in coding sequences
293 barcodes in Kl coding sequences
252 barcodes in Km coding sequences


In [48]:
df.to_csv(annotated_poolfile_path)

In [49]:
gff[gff['ID']=='kmYHR034C']

,scaffold,source,type,start,stop,strand,info,ID
8700,chr9,YGOB/YGAP,CDS,937252,938259,+,"gene_id ""KM020C04300""; transcript_id ""KM020C04300""",kmYHR034C
